<a href="https://colab.research.google.com/github/Irajkiran/appAIChatPDF/blob/main/appChatPDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install sentence_transformers
!pip install langchain-huggingface
!pip install PyPDF2
!pip install faiss-cpu
!pip install langchain-community


In [ ]:
import huggingface_hub
from huggingface_hub import login
from transformers import pipeline
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import PyPDF2
from PyPDF2 import PdfReader
import langchain
from huggingface_hub import login
from google.colab import userdata
import langchain_community
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter


def extract_text(pdf_path):
  """
  Extracts text content from a PDF document.
  """
  with open(pdf_path, 'rb') as f:
    reader = PdfReader(f)
    text = ""
    for page in reader.pages:
      text += page.extract_text()

  return text

def get_pdf_chunks(text):
  text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=200
    )

  chunks = text_splitter.split_text(text=text)
  return chunks

def get_vector_store(text_chunks):
  embeddings = SentenceTransformerEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  #embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-mpnet-base
  VectorStore = FAISS.from_texts(text_chunks,embeddings)
  VectorStore.save_local("faiss_index")

def get_conversational_chain():
  prompt_template = """
  Answer the question as detailed as possible from the provided context, make sure to provide all the details, if the answer is not in the
  provided context just say, "answer is not available in the context", don't provide the worng answer.\n
  Context: \n{context}?\n
  Question: \n{question}\n

  Answer:
  """
    # Load Gemma model from Hugging Face
  _tokenhuggungface = userdata.get('HuggingFaceToken')

  login(token = _tokenhuggungface)
  # Load model directly
  tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it",model_max_length=512)
  model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

  # Wrap the Hugging Face model in a pipeline for text generation
  pipe = pipeline(
      "text-generation", model=model, tokenizer=tokenizer, max_length=2048, truncation=True

  )

  #response = pipe(f"Document says: {text} \n You: {user_input}", truncation=True)[0]["generated_text"]

  # Create a HuggingFacePipeline object, which is a Runnable
  llm = HuggingFacePipeline(pipeline=pipe)

  prompt = PromptTemplate(template=prompt_template, input_variables=["context","question"])
  chain = load_qa_chain(llm,chain_type="stuff", prompt=prompt)

  # Extract text from PDF
  #text = extract_text(pdf_path)
  return chain

def user_input(user_question):
  embeddings = SentenceTransformerEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
  #db.save_local("faiss_index")
  new_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization = True)
  docs = new_db.similarity_search(user_question)

  chain = get_conversational_chain()


  response = chain(
      {"input_documents":docs, "question": user_question}
      , return_only_outputs = True)

  print(response)


def chat_with_pdf(pdf_path):
  """
  Initiates a text-based chat interaction with the PDF content.
  """

  get_vector_store(get_pdf_chunks(extract_text(pdf_path)))

  while True:
    # User query
    _user_input = input()
    if _user_input.lower() == "quit":
      break

    # Print response
    print(f"PDF: {user_input(_user_input)}")

if __name__ == "__main__":
  # Specify your PDF path
  pdf_path = r"/content/RPA.pdf"

  # Start chat interaction
  chat_with_pdf(pdf_path)

  print("Chat session ended.")